In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
url = 'https://imetech.com.my/blog/page/30/?et_blog'
r = requests.get(url)

In [3]:
soup = BeautifulSoup(r.content, "lxml")

In [4]:
a = soup.find_all('article')
a = [a_.find('a').get('href') for a_ in a]
a

['https://imetech.com.my/whats-new-2010-configuration-publisher/',
 'https://imetech.com.my/whats-new-2010-configuration-publisher/',
 'https://imetech.com.my/the-power-of-photoview-360-photoshop/',
 'https://imetech.com.my/answer-of-solidworks-word-puzzle/',
 'https://imetech.com.my/good-job-from-3dvia-composer/',
 'https://imetech.com.my/solidworks-education-edition/',
 'https://imetech.com.my/whats-new-in-solidworks-2010-virtual-component/',
 'https://imetech.com.my/solidworks-word-puzzle/',
 'https://imetech.com.my/events-lets-go-simulate/',
 'https://imetech.com.my/solidworks-floxpress/',
 'https://imetech.com.my/new-user-interface-of-solidworks-simulation-xpress/']

In [5]:
def generator():
    while True:
        yield
        
index = 1
all_a = []

for _ in tqdm(generator()):
    url = f'https://imetech.com.my/blog/page/{index}/?et_blog'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")

    a = soup.find_all('article')
    a = [a_.find('a').get('href') for a_ in a]
    
    if not len(a):
        break
        
    all_a.extend(a)
    
    index += 1

125it [06:26,  3.09s/it]


In [7]:
urls = sorted(list(set(all_a)))
len(urls)

486

In [12]:
url = urls[1]
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

In [13]:
soup.find('h1').text.strip()

'3 dudes gone 3D'

In [14]:
soup.find('div', {'class': 'entry-content'}).get_text(separator="\n")

'\n\nEverybody share, everybody win! This is the new day!\n\n\nLet me put it right, SolidWorks moving into movie industry? Hmm.. interesting..\n\n\nRecently SolidWorks Corp. post a new ‘movie’ into their website~\n \n3Dudes gone 3D\n.\n \n\n\n \n\n\nIt’s kinda funny but hey, this is what I like about SolidWorks, using Real innovative way to promote their products & educate us.\n\n\nBack to the movie, there is a part where they were discussing about SolidWorks enterprise PDM , two words appeared from the VP, words that so..so perfectly describe SolidWorks enterprise PDM~\n\n\nCollaboration efficiency!\n \n& I’m totally agree with it!\n\n\nI not going to share much about the video content, it will better if you can watch & enjoy it by yourself.\n\n\nDid I heard macrobating?\n \nClick me\n to watch all the available episodes.\n\n\n~Sew~\n\n'

In [18]:
data = []
for url in tqdm(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    try:
        data.append({
            'url': url,
            'title': soup.find('h1').text.strip(),
            'body': soup.find('div', {'class': 'entry-content'}).get_text(separator="\n"),
        })
    except:
        pass

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 486/486 [25:51<00:00,  3.19s/it]


In [19]:
len(data)

475

In [20]:
with open('imetech.com.my.jsonl', 'w') as fopen:
    for d in data:
        fopen.write(f'{json.dumps(d)}\n')